<a href="https://colab.research.google.com/github/taureandyernv/colabs/blob/master/airport_network_analysis_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using RAPIDS cuGraph and cuSpatial to analyze airport and flight data

## Intro
We have airports and flights datasets.  We have cuGraph and cuSpatial.  What craziness can we get up to here?

We're going to use cuGraph and cuSpatial to answer these questions of our data:
1. Which airport is the most trafficked airport in our dataset?
1. What are the max number of plane rides (hops) do you need to take to get from the most trafficked airport to get to any other airport in our dataset?
1. How many hops do you need to take to get from the most trafficked airport to one of the least trafficked airport?
1. How far is that distance really?
1. What is the topology of our airport network, based on our dataset and distance from one another?

Note: The Airports data in this toy dataset is using hashed identifiers. In the beginning, this may throw you for a loop, but by the end of the notebook everything will be clearer.

# Install RAPIDS
This takes a few minutes, as RAPIDS is currently not preinstalled in Colab.  First, make sure you're running a GPU instance of Colab.  Next, we'll check to see if you are running a P100 or T4 GPU.  Restart your instance if you are not.  Finally, we'll run the RAPIDS install script.

In [0]:
!nvidia-smi

In [0]:
# Install RAPIDS
!wget -nc https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

dist_package_index = sys.path.index("/usr/local/lib/python3.6/dist-packages")
sys.path = sys.path[:dist_package_index] + ["/usr/local/lib/python3.6/site-packages"] + sys.path[dist_package_index:]
sys.path
if os.path.exists('update_pyarrow.py'): ## This file only exists if you're using RAPIDS version 0.11 or higher
  exec(open("update_pyarrow.py").read(), globals())

## Imports and Data Gathering/Prep

In [0]:
import pandas as pd
import numpy as np
import cuspatial, cugraph, cudf, cuml

In [0]:
data_dir = ''
fdf = cudf.read_csv(data_dir+'flights.csv')
adf = cudf.read_csv(data_dir+'airports.csv')

In [0]:
fdf.head()

,PASSENGERS,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID
0,1,12264,10397
1,1,13541,12197
2,1,10599,12206
3,1,10792,10581
4,1,10792,14576


In [0]:
fdf.dtypes

PASSENGERS           int64
ORIGIN_AIRPORT_ID    int64
DEST_AIRPORT_ID      int64
dtype: object

### Prep
Since we'll be using cuGraph, which uses int32, and the above dtypes are int64, we recast each Series:

In [0]:
fdf['ORIGIN_AIRPORT_ID'] = fdf['ORIGIN_AIRPORT_ID'].astype(np.int32)
fdf['DEST_AIRPORT_ID'] = fdf['DEST_AIRPORT_ID'].astype(np.int32)
fdf['PASSENGERS'] = fdf['PASSENGERS'].astype(np.int32)

In [0]:
fdf.dtypes

PASSENGERS           int32
ORIGIN_AIRPORT_ID    int32
DEST_AIRPORT_ID      int32
dtype: object

Okay, better!  Now let's make some some graphs.  Why?  Cause graphs are fun and informative!

## Graphs
Recall that we're going to ask these questions of our data:
1. Which airport is the most trafficked airport in our dataset?
1. What are the max number of plane rides (hops) do you need to take to get from the most trafficked airport to get to any other airport in our dataset?
1. How many hops do you need to take to get from the most trafficked airport to one of the least trafficked airport?
1. How far is that distance really?
1. What is the topology of our airport network, based on our dataset and distance from one another?

**Let's get started!**

### Build the foundations

In [0]:
G = cugraph.Graph()
G.add_edge_list(fdf["ORIGIN_AIRPORT_ID"], fdf["DEST_AIRPORT_ID"])

/usr/local/lib/python3.6/site-packages/cugraph/structure/graph.py:157: UserWarning: add_edge_list will be deprecated in next release. Use from_cudf_edgelist instead
  Use from_cudf_edgelist instead')


In [0]:
# cudf uses the same formatting controls as Pandas!
pd.options.display.max_rows = 10

fdf["ORIGIN_AIRPORT_ID"].value_counts()

13930    12020
11292    10297
10397     9137
13487     6848
11433     6715
         ...  
16816        1
16823        1
16838        1
16839        1
16840        1
Name: ORIGIN_AIRPORT_ID, Length: 1147, dtype: int32

In [0]:
fdf["DEST_AIRPORT_ID"].value_counts()

13930    12082
11292    10425
10397     9179
13487     6913
11433     6651
         ...  
16819        1
16820        1
16838        1
16839        1
16840        1
Name: DEST_AIRPORT_ID, Length: 1181, dtype: int32

### Question 1: Which airport is the most trafficked airport in our dataset?

The easiest way to find out which airport is the most trafficked is the same way Google does it for websites: Pagerank!

In [0]:
df_page = cugraph.pagerank(G)

So now we have a graph, `df_page`.  Great!  What does it look like?

In [0]:
df_page.head()

,vertex,pagerank
0,0,0.000042
1,1,0.000042
2,2,0.000042
3,3,0.000042
4,4,0.000042


Pagerank isn't ordered by rank, but by vertex number, but it is easy to find the max rank and sort the orders.  Let's get our max and the top 10 airports in our dataset

In [0]:
pr_max = df_page['pagerank'].max()
print(pr_max)

0.0033526928164064884


In [0]:
sort_pr = df_page.sort_values('pagerank', ascending=True) # Just for fun, we're looking to see which airports have the least traffic
sort_pr.head(10)

,vertex,pagerank
0,0,0.000042
1,1,0.000042
2,2,0.000042
3,3,0.000042
4,4,0.000042
5,5,0.000042
6,6,0.000042
7,7,0.000042
8,8,0.000042
9,9,0.000042


In [0]:
sort_pr = df_page.sort_values('pagerank', ascending=False)
sort_pr.head(10)

,vertex,pagerank
12197,12197,0.003353
15167,15167,0.003242
10299,10299,0.002682
11292,11292,0.002270
11630,11630,0.002250
13930,13930,0.002179
11298,11298,0.001972
10693,10693,0.001860
10397,10397,0.001860
13487,13487,0.001843


Those are the top 10 trafficked airports.  While it was easy to see from the origin and destination airports counts that 13930 would be the most trafficked, the order of the others in the list required a bit more work.  It is also interesting that no single airport accounts for 1% of the total flights.

### Question 2: max number of plane rides (hops)?

Let's do a breadth first search (BFS) on the airports to fly out of and see how many hops it takes to get from popular airport, 13930, to an isolated one.  We'll do the BFS from the most poular airport to a randomly chosen one.

In [0]:
df = cugraph.bfs(G,13930)

In [0]:
df.count()

vertex         16841
distance       16841
predecessor    16841
dtype: int64

In [0]:
df['predecessor'].value_counts()

-1        15631
 13930      236
 10299       80
 11630       75
 12197       64
          ...  
 15579        1
 15779        1
 15855        1
 15919        1
 16101        1
Name: predecessor, Length: 232, dtype: int32

hmmm...what's `-1`?  Why does it's value so high?  Well, maybe it doesn't matter...let's get the max

In [0]:
df["distance"].max()

2147483647

**Whoa!**  That distance value is unexpected...but really not.  In the BFS demo, Brad told us that this occurs because the isolated vertex, 0, is unreachable.  Whenever a graph contains disjointed components, the distance to the unconnected vertices will always be max_int.  He also showed us how to fix it by dropping all insanely large distances.  We'll keep `df` untouched, in case we need it again, and make a second dataframe `df2`

In [0]:
# drop all large distances 
exp="distance < 100"
df2 = df.query(exp)

In [0]:
df2['predecessor'].value_counts()

13930    236
10299     80
11630     75
12197     64
11292     43
        ... 
15579      1
15779      1
15855      1
15919      1
16101      1
Name: predecessor, Length: 232, dtype: int32

That looks better!  A positive number has the most, and it's of course, airport 13930.  Now, let's see what the real graph distance is.

In [0]:
df2["distance"].max()

5

Okay great!  We know that no matter what, in the US, you're no more than 5 flights away from any other airport.  

### Question 3: How many hops do you need to take to get from the most trafficked airport to one of the least trafficed airports
Let's find out how many flights it takes to get us to a remote airport.  Let's pick one that has 1 flight from it.  I'm choosing `16498`, but you can change that value to another airport.  Also, there's a helper function to help make it a nicer print.

In [0]:
end_airport = 16498 # change to any other airport

In [0]:
def print_path(df, id):
    
    # Use the BFS predecessors and distance to trace the path 
    # from vertex id back to the starting vertex ( vertex 1 in this example)
    dist = df['distance'][id]
    lastVert = id
    route=[]
    route.append(lastVert)
    for i in range(dist):
        nextVert = df['predecessor'][lastVert]
        route.append(nextVert)
        d = df['distance'][lastVert]
        print("Airport " + str(lastVert) + " was reached from airport " + str(nextVert) + 
        " where the graph distance to Airport 13930 was " + str(d) )
        lastVert = nextVert
    return route

In [0]:
route = print_path(df, end_airport)

Airport 16498 was reached from airport 12478 where the graph distance to Airport 13930 was 2
Airport 12478 was reached from airport 13930 where the graph distance to Airport 13930 was 1


In [0]:
airports = adf.loc[adf.AIRPORT_ID.isin(route)]
print(airports)

      AIRPORT_ID   LATITUDE  LONGITUDE
2381       12478  40.638611 -73.776944
3770       13930  41.978056 -87.906111
6159       16498  42.913889 -76.440833


If you used my number, it would take 2 flights So now we know which airports you would connect to between those two airports.  But that is the graph distance.  What about the real distances?  

### Question 4:  How far is that distance really?
Well, for that, we need to bring in our other dataset, `adf`, which is a list of the airport's latitude and longitudes, as well as the GPU accelerated `cuSpatial` library to compute the Haversine distances (distances on the surface of the globe [sphere] instead of a straight line) 

In [0]:
adf.head()

,AIRPORT_ID,LATITUDE,LONGITUDE
0,10001,58.109444,-152.906667
1,10003,65.548056,-161.071667
2,10004,68.083333,-163.166667
3,10005,67.570000,-148.183889
4,10006,57.745278,-152.882778


Let's make a new function that calculates the haversine distance of all the airports in our flights at once.  This is a great time to use merge().  We'll do 2 merges, first on `ORIGIN_AIRPORT_ID` and then on `DEST_AIRPORT_ID`. To do the merge, we'll need to typecast the queries on our original 2 dataframes.

In [0]:
fdf['AIRPORT_ID'] = fdf['ORIGIN_AIRPORT_ID'].astype(np.int64) # create a common key with origin airport
hdf = fdf.merge(adf, on=['AIRPORT_ID'], how='left')
hdf.rename(columns = {'LATITUDE': 'LATITUDE_O', 'LONGITUDE': 'LONGITUDE_O'}, inplace=True) # Origin lat and long
hdf['AIRPORT_ID'] = hdf['DEST_AIRPORT_ID'].astype(np.int64) # recreate a common key with destination airport
hdf = hdf.merge(adf, on=['AIRPORT_ID'], how='left')
hdf.rename(columns = {'LATITUDE': 'LATITUDE_D', 'LONGITUDE': 'LONGITUDE_D'}, inplace=True) # Origin lat and long
hdf.head()

,PASSENGERS,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIRPORT_ID,LATITUDE_O,LONGITUDE_O,LATITUDE_D,LONGITUDE_D
0,6,15016,15454,15454,38.747222,-90.364444,39.944167,-91.197222
1,6,10800,13796,13796,34.200000,-118.357778,37.721389,-122.220833
2,6,10327,12892,12892,38.212222,-122.280000,33.942500,-118.408056
3,6,12211,14057,14057,35.972778,-115.134444,45.589167,-122.595000
4,6,13487,12451,12451,44.886111,-93.217778,30.484722,-81.687778


In [0]:
x1 = hdf["LONGITUDE_O"]
y1 = hdf["LATITUDE_O"]
x2 = hdf["LONGITUDE_D"]
y2 = hdf["LATITUDE_D"]

hdf['H-distance'] = cuspatial.haversine_distance(x1, y1, x2, y2)
hdf.head(10)

,PASSENGERS,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIRPORT_ID,LATITUDE_O,LONGITUDE_O,LATITUDE_D,LONGITUDE_D,H-distance
0,6,15016,15454,15454,38.747222,-90.364444,39.944167,-91.197222,151.134493
1,6,10800,13796,13796,34.200000,-118.357778,37.721389,-122.220833,523.538699
2,6,10327,12892,12892,38.212222,-122.280000,33.942500,-118.408056,588.499423
3,6,12211,14057,14057,35.972778,-115.134444,45.589167,-122.595000,1238.847332
4,6,13487,12451,12451,44.886111,-93.217778,30.484722,-81.687778,1891.376982
5,6,14689,13796,13796,34.427778,-119.839444,37.721389,-122.220833,424.143374
6,6,12455,12992,12992,35.830000,-90.648333,34.729167,-92.234444,188.972691
7,6,10721,12892,12892,42.363611,-71.006111,33.942500,-118.408056,4192.963746
8,6,13541,10800,10800,41.392778,-70.616667,34.200000,-118.357778,4216.043452
9,6,14908,11259,11259,33.675556,-117.866944,32.846944,-96.853333,1952.632867


Let's get the actual distances that one must fly to get between those airports

In [0]:
H = cugraph.Graph()
#hdf["ORIGIN_AIRPORT_ID_0"] = hdf["ORIGIN_AIRPORT_ID"] - 10001
#hdf["DEST_AIRPORT_ID_0"] = hdf["DEST_AIRPORT_ID"] - 10001
#hdf["data"] = 1.0
H.add_edge_list(hdf["ORIGIN_AIRPORT_ID"], hdf["DEST_AIRPORT_ID"], hdf["H-distance"])
hgdf = cugraph.bfs(H,13930)

/usr/local/lib/python3.6/site-packages/cugraph/structure/graph.py:157: UserWarning: add_edge_list will be deprecated in next release. Use from_cudf_edgelist instead
  Use from_cudf_edgelist instead')


**Fun Fact** Deleting the -1s throws off your indexes and doesn't return you a valid answer.  Try it if you'd like!

In [0]:
def print_dist_path(df, id):
    # Use the BFS predecessors and distance to trace the path 
    # from vertex id back to the starting vertex ( vertex 1 in this example)
    dist = df['distance'][id]
    hdist = 0
    print("Your overall flight has " + str(dist) + " hops")
    lastVert = id
    for i in range(dist):
        nextVert = df['predecessor'][lastVert]
        d = df['distance'][lastVert]
        a = hdf.query("ORIGIN_AIRPORT_ID == @nextVert and DEST_AIRPORT_ID == @lastVert")
        a.head()
        hdist = hdist+ a["H-distance"][0]
        print("Airport: " + str(lastVert) + " was reached from Airport " + str(nextVert) + 
        " and flight distance was " + str(a["H-distance"][0]) )
        lastVert = nextVert
    print("Your total flying distance was " + str(hdist))

In [0]:
print_dist_path(hgdf, 16838)

Your overall flight has 3 hops
Airport: 16838 was reached from Airport 13418 and flight distance was 235.3294186784242
Airport: 13418 was reached from Airport 12197 and flight distance was 416.77900332448866
Airport: 12197 was reached from Airport 13930 and flight distance was 1184.955067107391
Your total flying distance was 1837.0634891103039


Okay, pretty cool.  We now know the distance between these airports...but where are they in the world?  Normally, we'd use use [cuDataShader](https://github.com/rapidsai/cuDataShader) for this, but it is not a library in this container.  [They've got a great example here that you can adapt to your needs](https://github.com/rapidsai/cuDataShader/blob/master/cudatashader-notebooks/cuDatashader%20Edge%20Bundling%20(US%20air%20traffic).ipynb)

### Question 5: What is the topology of our airport network

Let's look at the topology of this network of airports.  One way to do that is to measure the modularity of our airport system!  To do that, we use Louvain.  However, we need to make some changes to our data, as Louvain requires us to start from 0.  It also requires weights.  Let's see how weights change our answer.  We will use our Haversine distances as our weights in one set, and be unweighted in the next!

In [0]:
L = cugraph.Graph()
L2 = cugraph.Graph()
hdf["ORIGIN_AIRPORT_ID_0"] = hdf["ORIGIN_AIRPORT_ID"] - 10001
hdf["DEST_AIRPORT_ID_0"] = hdf["DEST_AIRPORT_ID"] - 10001
hdf["data"]= 1.0
L2.add_edge_list(hdf["ORIGIN_AIRPORT_ID_0"], hdf["DEST_AIRPORT_ID_0"], hdf["data"]) # Unweighted Modularity
L.add_edge_list(hdf["ORIGIN_AIRPORT_ID_0"], hdf["DEST_AIRPORT_ID_0"], hdf["H-distance"]) # Distance Weighted Modularity

/usr/local/lib/python3.6/site-packages/cugraph/structure/graph.py:157: UserWarning: add_edge_list will be deprecated in next release. Use from_cudf_edgelist instead
  Use from_cudf_edgelist instead')


In [0]:
# Call Louvain on the graph
hgdf, mod = cugraph.louvain(L) 
hgdf2, mod2 =cugraph.louvain(L2) 
# Print the modularity score
print('Modularity using Distance as a weight was {}'.format(mod))
print()
print('Modularity unweighted was {}'.format(mod2))
print()

Modularity using Distance as a weight was 0.06543180748186618

Modularity unweighted was 0.19843214099425677



In [0]:
hgdf.head(10)

,vertex,partition
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,47
6,6,5
7,7,6
8,8,148
9,9,710


In [0]:
hgdf2.head(10)

,vertex,partition
0,0,0
1,1,1
2,2,2
3,3,3
4,4,1323
5,5,3033
6,6,4
7,7,5
8,8,3033
9,9,4442


That's a high partition number for both graphs.  This is of course, based on a small dataset of flights.  I'll be working on a larger one in notebooks_contrib that uses DOT 2015 Flight data and use cuDataShader for graph visualizations.  Let's see what the value counts look like.

In [0]:
print(len(hgdf['partition'].unique()))
print(len(hgdf2['partition'].unique()))

6055
5715


In [0]:
hgdf['partition'].value_counts()

2608    100
260      84
507      66
2891     47
1910     38
       ... 
6050      1
6051      1
6052      1
6053      1
6054      1
Name: partition, Length: 6055, dtype: int32

In [0]:
hgdf2['partition'].value_counts()

4442    645
4012    150
1323     49
4343     43
4711     42
       ... 
5710      1
5711      1
5712      1
5713      1
5714      1
Name: partition, Length: 5715, dtype: int32

It seems that the unweighted graph has fewer and larger communities (which makes sense).  Let's remove partitions of 1 from our assessment and get a full count of how many communities each graph has.  1 is too lonely of a number.

In [0]:
def get_mod(df):
    val_counts = df['partition'].value_counts()
    relevant_partitions = val_counts[val_counts>1].index
    print(len(relevant_partitions))
    query = 'partition == '+ str(relevant_partitions[0])
    for i in range (1, len(relevant_partitions)):
            query += ' or partition == '+ str(relevant_partitions[i])
    return df.query(query)

In [0]:
# How many partitions where found
def get_partitions(df):
    parts=[]
    part_ids = df["partition"].unique()
    for p in range(len(part_ids)):
        part = []
        for i in range(len(df)):
            #print(df['partition'][i])
            if (df['partition'][i] == part_ids[p]):
                part.append(df['vertex'][i] +1+10001)
        print("Partition " + str(part_ids[p]) + " contains these airports:")
        print(part)
        parts.append(part)
    return parts

In [0]:
print("Number of partitions > 1 in Distance Weighted Modularity:")
hgdf_1 = get_mod(hgdf)
print("Number of partitions > 1 in Unweighted Modularity:")
hgdf_2 = get_mod(hgdf2)

hgdf_1.head()

Number of partitions > 1 in Distance Weighted Modularity:
72
Number of partitions > 1 in Unweighted Modularity:
31


,vertex,partition
5,5,47
8,8,148
9,9,710
15,15,1920
16,16,47


In [0]:
print("------Distance Weighted Modularity------")
parts_hgdf_1 = get_partitions(hgdf_1)
print("------Unweighted Modularity------")
parts_hgdf_2 = get_partitions(hgdf_2)

------Distance Weighted Modularity------
Partition 47 contains these airports:
[10007, 10018, 10043, 10044, 10057, 10064, 10079, 10101, 10244, 10279, 11514, 12510, 12706, 12713, 12744, 12786, 12870, 13864, 13889, 14632, 15092, 15837]
Partition 136 contains these airports:
[10158, 10172, 13288]
Partition 143 contains these airports:
[10166, 12322, 12756, 14739]
Partition 148 contains these airports:
[10010, 10048, 10171, 11300, 11337, 13715, 14856, 14920, 15733]
Partition 161 contains these airports:
[10186, 10435, 11790, 12056, 12157, 13231, 14198, 14260]
Partition 192 contains these airports:
[10222, 10926, 11199, 13201, 14661]
Partition 195 contains these airports:
[10225, 15326]
Partition 212 contains these airports:
[10246, 10927, 11228, 11393, 11493, 11556, 11998, 12172, 12176, 12611, 12636, 12642, 12709, 12721, 12729, 12780, 13004, 14469, 15449]
Partition 260 contains these airports:
[10226, 10238, 10248, 10300, 10314, 10381, 10541, 10552, 10616, 10755, 10784, 10918, 11004, 11054

Okay great!  If you want to see these communities printed again, or even plot them with matplotlib, here you go.  Otherwise, we've answered our 5 questions.   Hooray!

## More Notebooks
Just remember, that while Colab is free, most of your time spent is waiting for RAPIDS to install.   The time to insights is much faster if RAPIDS is already installed.  If you have provisioned a machine with the necessary hardware and RAPIDS installed, you can check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib


In [0]:
#print(parts_hgdf_1)

In [0]:
#print(parts_hgdf_2)